# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

<h2 style="color: black;">
    <span style="background-color: #ECBE7B; padding: 1px;">&ensp;Part 1: Conic Linear Programs&ensp;</span>
</h2>

In [3]:
import Pkg
Pkg.activate("..")

## these lines are required for local installations
Pkg.develop(path="../../SCPToolbox.jl/")
Pkg.precompile()

using SCPToolbox
using ECOS
# using Ipopt
# using SCS
# using OSQP

  Activating project at `c:\Users\julia\OneDrive - Massachusetts Institute of Technology\Documents\Grad School\NSTGRO\Solver Development\SCPToolbox_tutorial`
   Resolving package versions...
   Installed SCS_GPU_jll ────── v3.2.0+0
   Installed OSQP_jll ───────── v0.600.200+0
   Installed DiffRules ──────── v1.11.0
   Installed NaNMath ────────── v1.0.0
   Installed Parsers ────────── v2.3.1
   Installed SCS ────────────── v1.1.1
   Installed StaticArrays ───── v1.4.4
   Installed Tokenize ───────── v0.5.23
   Installed SCS_jll ────────── v3.2.0+0
   Installed LogExpFunctions ── v0.3.14
   Installed OSQP ───────────── v0.8.0
   Installed JuliaFormatter ─── v0.22.10
   Installed ForwardDiff ────── v0.10.27
   Installed CEnum ──────────── v0.4.2
   Installed MathOptInterface ─ v1.2.0
    Updating `C:\Users\julia\OneDrive - Massachusetts Institute of Technology\Documents\Grad School\NSTGRO\Solver Development\SCPToolbox_tutorial\Project.toml`
  [5ae59095] + Colors v0.12.8
  [e2685f51] + EC

__General form__:

\begin{equation*}
\begin{aligned}
\underset{x\in\mathbb{R}^n}{{\text{minimize}}} &~ c^\top x \\
\text{subject to} &~Ax+b \in \mathbb{K}
\end{aligned}
\end{equation*}

where $c\in\mathbb{R}^n,~b\in\mathbb{R}^m,~A\in\mathbb{R}^{m\times n}$ are problem parameters, and $\mathbb{K}$ is a cartesian product of convex cones. 

- The optimization problem shown can be solved using state-of-the-art solvers like `ECOS`, `Gurobi` and `Mosek`. 

- They typically need a parser to convert the problem data to a canonical form. 

- Such solvers are not user-friendly to interface with.

- In `SCPToolbox.jl`, henceforth referred to as _the toolbox_, a conic linear program is formulated by instantiating a `ConicProblem` object with the choice of solver and its associated options.  

The cones supported by the toolbox are described in the table below:

|    Label        | Notation     |     Name      | Definition |
|:---------------:|:----:|:-------------:|:----------:|
|`UNCONSTRAINED`  |  $\mathbb{K}_{ \lesseqgtr}$ | Unconstrained cone  |  $$\{x\in\mathbb{R}^n\}$$ |
|`ZERO`           |  $\mathbb{K}_{=0}$ | Zero cone             | $\{x\in\mathbb{R}^n\mid x=0\}$                               |
|`NONPOS`         |  $\mathbb{K}_{\leq 0}$ | Non-positive orthant  | $\{x\in\mathbb{R}^n\mid x\leq 0\}$                           |
|`L1`             |  $\mathbb{K}_{1}$ | $l_1$ norm cone       | $\{(t,x)\in\mathbb{R}\times\mathbb{R}^n\mid \lVert x\rVert_1\leq t\}$ |
|`SOC`             |  $\mathbb{K}_{2}$ | $l_2$ norm cone       | $\{(t,x)\in\mathbb{R}\times\mathbb{R}^n\mid \lVert x\rVert_2\leq t\}$ |
| `LINF` |  $\mathbb{K}_{\infty}$ |  $l_\infty$ norm cone |  $\{(t,x)\in\mathbb{R}\times\mathbb{R}^n\mid \lVert x\rVert_\infty\leq t\}$  |
| `GEOM` |  $\mathbb{K}_{\text{GM}}$ | Geometric mean cone | $ \big\{ (t,x)\in\mathbb{R}\times\mathbb{R}^n\mid \left(\prod_{i=1}^n x_i\right)^{1/n} \geq t \big\}$ |
| `EXP` |  $\mathbb{K}_{\text{EXP}}$ | Exponential cone | $\{(x,y,z)\in \mathbb{R}\times \mathbb{R} \times \mathbb{R} \mid y\,\text{exp}(x/y) \leq z,~y>0\}$ |

In [4]:
opts = Dict("verbose" => 0)
pbm = ConicProgram(solver = ECOS, solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


## Variables

Variables are the quantities that the SCP Toolbox determines values for upon solution. These variables therefore begin as symbolic values during problem formulation, but are ultimately given numerical values upon convergence to a locally optimal trajectory solution. 

Variables are generally broken up into three main categories: 
- **scalars**: variables containing a single element.  *e.g.* $x \in \mathbb{R}$.
- **vectors**: variables containing a single row or column.  *e.g.* $x \in \mathbb{R}^N$.
- **matrices**: variables for both rows and columns greater than size two.  *e.g.* $x \in \mathbb{R}^{N \times M}$.

### Variable Properties


Each variable declared by the user is defined by a set of properties.

1. **Elements:** The number of elements in the variable, given as a scalar.
2. **Shape:** The tuple defining the dimensions of the variable, given as $(N,M)$ for an $N \times M$-dimensional matrix.
3. **Name:** The title given as a user-defined string.
4. **Block index:** The position where the variable is stored in the stack after the parser reformats all variable data into a single vector, where the data within each element of the variable is stored in a corresponding block of elements. Matrices are vectorized in column-major form.
5. **Indices:** The elements of the stack where the variable data is stored, making up the block.
6. **Type:** Datatype.
7. **Scaling Type:** If a variable is scaled, then the physical quantity is nondimensionalized with an affine mapping when exposed to the optimizer.
8. **Value Type:** The symbolic or numerical quantity assigned to a given variable.

### Variable Declaration

At the high level, each variable declaration is done with the `@new_variable()` function. However, this constructor is overloaded such that variables may be declared with a variety of different input-argument syntaxes. Each call to the constructor takes in the corresponding problem that they are associated with alongside other input arguments.

A scalar variable may be declared via passing the corresponding problem, a dimension of 1, and a variable name as constructor inputs. Here, scaling is not applied. By default, a scalar variable is treated as a vector variable of length one, so be mindful of the vector-scalar operations.

A scalar quantity may be simply defined as follows, without specifying a dimension.

In [5]:
q = @new_variable(pbm, "q")

Vector variable
  1 elements
  (1,) shape
  Name: q
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     q

A vector variable (with dimension $>1$) may be declared as follows. Note that the value now has multiple elements.

In [6]:
v = @new_variable(pbm, 3, "v")

Vector variable
  3 elements
  (3,) shape
  Name: v
  Block index in stack: 2
  Indices in stack: 2:4
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     v[1]
     v[2]
     v[3]

A matrix variable (with row and column dimensions both $>1$) may be declared as follows.

A matrix variable may represent a standard matrix, such as those commonly applied to achieve a linear transformations on a vector. A matrix variable may also represent a set of vectors, such as values of the state or control from a dynamic system, sampled at variable discrete timesteps along a trajectory.

In [7]:
y = @new_variable(pbm, (2, 5), "y")

Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 3
  Indices in stack: 5:14
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
     y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]

Individual row or column vectors of a matrix may be indexed as follows. Note that these indexed quantities are themselves vector variables.

In [8]:
y[:, 1]

Vector variable
  2 elements
  (2,) shape
  Name: y
  Block index in stack: 3
  Indices in stack: 5:6
  Type: SubArray{JuMP.AffExpr, 1, Matrix{JuMP.AffExpr}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     y[1,1]
     y[2,1]

### Variable Operations


### Scaling

Scaling is an important operation which improves numerical conditioning of the underlying optimization problem by applying an affine mapping to the physical quantity in order to produce a corresponding scaled quantity that lies approximately between $0$ and $1$. 

The utility of the affine mapping becomes clear while defining simple set constraints on the value of the variable. Examples of such constraints are a ball, halfspace, or box, etc. This is discussed in more detail in the Constraints section below.

In the scalar case, this affine mapping may be approximated as follows for physical quantity $x$ and scaled quantity $\hat{x}$: 

$x = a \hat{x} + b$, 

where $a = (x_\text{max} - x_\text{min})$ and $b = x_\text{min}$.

The affine mapping for scaling a given physical variable is called via the `@scale()` function. This function is given the vector to be scaled, a vector of entries that will linearly scale each element of the variable, and an affine offset, respectively.

In [9]:
p = @new_variable(pbm, "p")
@scale(p, 10, 5);

The value of this new affine mapping can be queried with the `value()` function as follows:

In [10]:
value(p)

1-element Vector{JuMP.AffExpr}:
 10 p + 5

To apply a different scaling to each element of a variable, diagonal entries of a matrix that scales each variable element may be defined. In addition, vector-value affine offsets may also be applied. 

In [11]:
@scale(y, [2; 4], [3;1])

In [12]:
y

Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 3
  Indices in stack: 5:14
  Type: Matrix{JuMP.AffExpr}
  Affine scaling x=(S.*xh).+c
  Any perturbation allowed
  Value =
     y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
     y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]

The value of this scaled matrix variable is given below.

In [13]:
value(y)

2×5 Matrix{JuMP.AffExpr}:
 2 y[1,1] + 3  2 y[1,2] + 3  2 y[1,3] + 3  2 y[1,4] + 3  2 y[1,5] + 3
 4 y[2,1] + 1  4 y[2,2] + 1  4 y[2,3] + 1  4 y[2,4] + 1  4 y[2,5] + 1

The variables created for a given problem are displayed.

In [14]:
pbm

Conic linear program

  Feasibility problem
  15 variables (4 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    15 elements
    4 blocks
     1) 1    ... q
     2) 2:4  ... v
     3) 5:14 ... y
     4) 15   ... p

  Parameter argument
    0 elements
    0 blocks


Finally, the variables of this problem can be queried explicitly.

In [15]:
variables(pbm)

4-element Vector{SCPToolbox.Parser.ConicLinearProgram.VariableArgumentBlock}:
 Vector variable
  1 elements
  (1,) shape
  Name: q
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     q
 Vector variable
  3 elements
  (3,) shape
  Name: v
  Block index in stack: 2
  Indices in stack: 2:4
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     v[1]
     v[2]
     v[3]
 Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 3
  Indices in stack: 5:14
  Type: Matrix{JuMP.AffExpr}
  Affine scaling x=(S.*xh).+c
  Any perturbation allowed
  Value =
     y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
     y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]
 Vector variable
  1 elements
  (1,) shape
  Name: p
  Block index in stack: 4
  Indices in stack: 15
  Type: Vector{JuMP.AffExpr}
  Affine scaling x=(S.*xh).+c
  Any perturbation allowed
  Value =
     p

## Constraints

A conic constraint $y\in\mathbb{K}$ (say for $\mathbb{K} = \mathbb{K}_2$) can be modelled in the toolbox as follows:

In [16]:
pbm = ConicProgram(solver = ECOS, solver_options = Dict("verbose" => 0))

x = @new_variable(pbm, 4, "x")
t = @new_variable(pbm, "t")

cstr = @add_constraint(pbm, SOC, "my-soc", (x, t) -> vcat(100*t, x))

Name: my-soc
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  100 t
  x[1]
  x[2]
  x[3]
  x[4]
Affine function
Arguments:
  x (block 1) : 1:4
  t (block 2) : 5


In [17]:
variables(pbm,"^x\$") == x

true

Note the use of [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) for searching though the list of variables names.

In [18]:
constraints(pbm, "^my-soc")

Name: my-soc
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  100 t
  x[1]
  x[2]
  x[3]
  x[4]
Affine function
Arguments:
  x (block 1) : 1:4
  t (block 2) : 5


## Constraint Examples

Many simple constraint sets in trajectory optimization problems like box, ball, halfspace and hyperplane can be represented as conic constraints.

### Box

A box with bounds $l,u\in\mathbb{R}$ with $l<u$ can be formulated as as non-positive orthant cone:

\begin{equation*}
\{x\in\mathbb{R}^n~|~l\leq x \leq u\}
\end{equation*}

\begin{equation*}
l\leq x \leq u \iff \left[ \begin{array}{c}I\\ -I\end{array}\right]x + \left[\begin{array}{c}-u\mathbf{1}\\ l\mathbf{1} \end{array}\right] \in \mathbb{K}_{\leq 0}
\end{equation*}

In [17]:
l = 2
u = 9

A = [1  0; 0 1; -1 0; 0 -1]
b = [-u; -u; l; l]

pbm = ConicProgram()

x = @new_variable(pbm, 2, "x")

cstr = @add_constraint(pbm, NONPOS, "my-box", x -> A*x .+ b)

Name: my-box
Cone f(x,p)∈K, where:
K is a nonpositive orthant cone, {z : z≤0}
f(x,p) = 
  x[1] - 9
  x[2] - 9
  -x[1] + 2
  -x[2] + 2
Affine function
Arguments:
  x (block 1) : 1:2


### Ball

A ball with radius $r \in \mathbb{R}_+$ set can be represented as a $l_2$ norm cone (second-order cone):

\begin{equation*}
\{x\in\mathbb{R}^n~|~\|x\|_2\leq r\}
\end{equation*}

\begin{equation*}
\|x\|_2\leq r \iff \left[ \begin{array}{c} 0 \\ I \end{array} \right] x + \left[ \begin{array}{c} r \\ 0 \end{array} \right]\in \mathbb{K}_2
\end{equation*}

In [18]:
r = 4

A = [0  0; 1 0; 0 1]
b = [r;0;0]

pbm = ConicProgram()

x = @new_variable(pbm, 2, "x")

cstr = @add_constraint(pbm, SOC, "my-ball", x -> A*x .+ b)

Name: my-ball
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  4
  x[1]
  x[2]
Affine function
Arguments:
  x (block 1) : 1:2


### Hyperplane

A hyperplane defined by $a\in \mathbb{R}^n$ and $b\in\mathbb{R}$ can be represented as a zero cone:

\begin{equation*}
\left\{x\in\mathbb{R}^n~|~a^\top x = b\right\}
\end{equation*}

\begin{equation*}
a^\top x = b \iff \left[a^\top\right]x + (-b) \in \mathbb{K}_{=0}
\end{equation*}

In [19]:
A = [1.1 3.2]
b = -9

pbm = ConicProgram()

x = @new_variable(pbm, 2, "x")

cstr = @add_constraint(pbm, ZERO, "my-hypln", x -> A*x .+ b)

Name: my-hypln
Cone f(x,p)∈K, where:
K is a zero cone, {z : z=0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2


### Halfspace

A halfspace defined by $a\in \mathbb{R}^n$ and $b\in\mathbb{R}$ can be represented as a non-positive orthant cone:

\begin{equation*}
\left\{x\in\mathbb{R}^n~|~a^\top x \leq b\right\}
\end{equation*}

\begin{equation*}
a^\top x \leq b \iff \left[a^\top\right] x + (-b) \in \mathbb{K}_{\leq 0}
\end{equation*}

In [20]:
A = [1.1 3.2]
b = -9

pbm = ConicProgram()

x = @new_variable(pbm, 2, "x")

cstr = @add_constraint(pbm, NONPOS, "my-hlfspc", x -> A*x .+ b)

Name: my-hlfspc
Cone f(x,p)∈K, where:
K is a nonpositive orthant cone, {z : z≤0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2


## Objective function

 - Regardless of whether optimization problems have constraints, the goal is to maximize or minimize a function i.e. the _objective_ or _cost_ function.

 - For e.g. propellant consumption and control effort are important metrics for rocket-powered planetary landing missions. We may also want to minimize the total energy to go from one point to another.

__Note__: Maximizing a reward is equivalent to minimizing its negative. 

Define a scalar variable, `x`. 

Define objective function, which we will henceforth refer to as simply "cost".

By default, the cost is initialized to zero when a new problem is generated. We use the `@add_cost` function to add terms to the cost `obj`.

 - The first argument indicates the `ConicProgram` instance `pbm` to which a cost term is added.

 - The second argument defines the cost function mapping in terms of the the input variable.

In [21]:
my_pars = Dict("a" => 1, "b" => 2.5)

opts = Dict("verbose" => 0)
pbm = ConicProgram(my_pars; solver = ECOS, solver_options = opts)

x = @new_variable(pbm, 2, "x")
t = @new_variable(pbm, "t")

@add_constraint(pbm, NONPOS, x -> -x*cst["b"])
@add_constraint(pbm, SOC, (x,t) -> vcat(t,x))
@add_constraint(pbm, NONPOS, (x,t) -> x .- [t;t] .+ cst["a"]*cst["b"])

@add_cost(pbm, t -> cst["a"]*t);

## Solve problem

With everything we have learned so far, we are now ready to solve a convex optimization problem!

In [22]:
exit_status = solve!(pbm)

OPTIMAL::TerminationStatusCode = 1

The optimal cost value is `J_opt`.

In [23]:
J_opt = objective_value(pbm)

2.499999998686782

The optimal values of the decision variables are `x_opt` and `t_opt`.

In [24]:
x_opt = value(x)

2-element Vector{Float64}:
 -5.973286724406856e-10
 -5.973287215778078e-10

In [25]:
t_opt = value(t)

1-element Vector{Float64}:
 2.499999998686782

The optimal values of the decision variables can also be accessed as mentioned before.

In [26]:
value(variables(pbm, "^x\$"))

2-element Vector{Float64}:
 -5.973286724406856e-10
 -5.973287215778078e-10

In [27]:
value(variables(pbm, "^t\$"))

1-element Vector{Float64}:
 2.499999998686782

Congratulations! 

We now know how to solve convex optimization problems, which form the basis for Sequential Convex Programming (SCP) problems, which we shall now get into with some exciting examples!